In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime 
import requests
import math
import re

from bs4 import BeautifulSoup

In [ ]:
# url do site
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# dicionario que diz para i end point que quem faz a requisicao e um chorme (Browser)
headers ={''}
headers = headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# Requisita essa url do servidor do site H&M
page = requests.get( url, headers=headers )

# Product lista das primeiras coletas id, cat, name, price
soup = BeautifulSoup( page.text, 'html.parser' )

# Toda minha vitrine
products = soup.find('ul', class_= 'products-listing small')
# Todos meu predutos da vitrine e sua cat, onde filtro todas que preciso
product_list = products.find_all('article', class_ = 'hm-product-item')

In [ ]:
# ========================================= product =========================================

# Product_id
product_list[0].get('data-articlecode')
product_id = [i.get('data-articlecode') for i in product_list]

# Product_category
product_list[0].get('data-category')
product_category = [i.get('data-category') for i in product_list]

# product_name
product_list = products.find_all('a', class_= 'item-link' )
product_list[0].get('title')
product_name = [i.get('title') for i in product_list]

# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [i.get_text() for i in product_list]


# Criando o data frame
data = pd.DataFrame([ product_id, product_name, product_category, product_price ]).T
#colunas do DataFrame
data.columns = ['product_id', 'product_name', 'product_category', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S:')
# plotar

In [ ]:

# =================== Explicacao de paginacao =============================================

# Pegando o total de produtos 36/83
total_iten = soup.find_all( 'h2', class_='load-more-heading' )[0].get('data-total')
total_iten
# como é 'STR' transformo em int 
print( int(total_iten))
# faco a conta de quantas page preciso para pegar o total de produto ou tds os dados
print(int(total_iten) / 36)
# Passo um math.ceil para arredondar qual sera o num de page
print(math.ceil(int(total_iten) / 36))
# Qtd de pag que eu preciso
page_number = math.ceil( int( total_iten ) / 36 )

# Resumo: pego o num toda de intem que tenho divido por 36 (vitrine) que é o num de produto que tenho em cada pag (paginaçao)
# e com o result da minha url02 eu vou encontar o numero de page_size que eu preciso 108 = preciso de 3 pag  de 36 prod
## Ex: se no caso tivesse iten 200 = (200 / 36) = resltado é num de pag . result * 36 = tamanho da vitrine para url

# Montado a url
url02 = url + '?pagesize=' + str( int( page_number*36 ) ) 
url02

In [137]:

url = 'https://www2.hm.com/en_us/productpage.0985159001.html'

# dicionario que diz para i end point que quem faz a requisicao e um chorme (Browser)
headers ={''}
headers = headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# Requisita essa url do servidor do site H&M
page = requests.get( url, headers=headers )

# ================  One Product =================================

# para acesar a pagina html do page , objeto da classe request
page.text
# Parametro de entrada para o beautifulsoup
soup = BeautifulSoup( page.text, 'html.parser' )

# ========================== Color Name ===============================
# Coletar a cor color_name
product_list = soup.find_all('a', class_='filter-option miniature')
product_color = [i.get('data-color') for i in product_list]

# product_id
product_id = [i.get('data-articlecode') for i in product_list]

# Ccriando o data frame
df_color = pd.DataFrame([product_id, product_color]).T
df_color.columns = ['product_id', 'color_name']

# # generate style id+ color id
df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

# =============================== Caategory =======================================

# Encontre todos os elementos que representam os produtos
product_elements = soup.find_all('div', class_='content pdp-text pdp-content')

# Inicialize uma lista para armazenar as informações de todos os produtos
all_products_info = []

for product_element in product_elements:
    product_info = {}
    
    # Extrair informações específicas para cada produto
    fit_element = product_element.find('dt', string='Fit')
    if fit_element:
        fit = fit_element.find_next('dd').text.strip()
        product_info['Fit'] = fit
        
    size_element = product_element.find('dt', string='Size')
    if size_element:
        size = size_element.find_next('dd').text.strip()
        product_info['Size'] = size    
    
    composition_element = product_element.find('h3', string='Composition')
    if composition_element:
        composition = composition_element.find_next('p').text.strip()
        product_info['Composition'] = composition
       
    pocket_lining_element = product_element.find('h4', string='Pocket lining')
    if pocket_lining_element:
        pocket_lining = pocket_lining_element.find_next('p').text.strip()
        product_info['Pocket lining'] = pocket_lining
        
    article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
    if article_number_element:
        article_number_text = article_number_element.text.strip()
        article_number = re.search(r'\d+', article_number_text).group()
        product_info['Article number'] = article_number    
        
    # Adicionar as informações do produto à lista
    all_products_info.append(product_info);

# criar data frame 
df_composition = pd.DataFrame(all_products_info)

df_composition['style_id'] = df_composition['Article number'].apply( lambda x: x[:-3])
df_composition['color_id'] = df_composition['Article number'].apply( lambda x: x[-3:])

# ========================== df_color + df_composition ================================== 
data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id');

# Requisicao para toda a vitrine

In [160]:
# dicionario que diz para i end point que quem faz a requisicao e um chorme (Browser)
headers ={''}
headers = headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

df_final = pd.DataFrame()

# cria a lista vazia para colocar as col que nao tenho aqui coletada caso meu novo prod tenha alguma caracteristca que nao peguei no promeiro
aux = []

cols = ['Article number','Composition','Fit','Pocket lining','Size','color_id','style_id']
df_patter = pd.DataFrame(columns=cols)

for i in range( len( data ) ):
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    print(url)
    # Requisita essa url do servidor do site H&M
    page = requests.get( url, headers=headers )
    
    # Parametro de entrada para o beautifulsoup
    soup = BeautifulSoup( page.text, 'html.parser' )

    # ========================== Color Name ===============================
    # Coletar a cor color_name
    product_list = soup.find_all('a', class_='filter-option miniature')
    product_color = [i.get('data-color') for i in product_list]

    # product_id
    product_id = [i.get('data-articlecode') for i in product_list]

    #    Ccriando o data frame
    df_color = pd.DataFrame([product_id, product_color]).T
    df_color.columns = ['product_id', 'color_name']

    # # generate style id+ color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

    # =============================== Caategory =======================================

    # Encontrei todos os elementos que representam os produtos
    product_elements = soup.find_all('div', class_='content pdp-text pdp-content')

    # Inicializei uma lista para armazenar as informações de todos os produtos
    all_products_info = []

    for product_element in product_elements:
        product_info = {}
    
        # Extrair informações específicas para cada produto
        fit_element = product_element.find('dt', string='Fit')
        if fit_element:
            fit = fit_element.find_next('dd').text.strip()
            product_info['Fit'] = fit
        
        size_element = product_element.find('dt', string='Size')
        if size_element:
            size = size_element.find_next('dd').text.strip()
            product_info['Size'] = size    
    
        composition_element = product_element.find('h3', string='Composition')
        if composition_element:
            composition = composition_element.find_next('p').text.strip()
            product_info['Composition'] = composition
       
        pocket_lining_element = product_element.find('h4', string='Pocket lining')
        if pocket_lining_element:
            pocket_lining = pocket_lining_element.find_next('p').text.strip()
            product_info['Pocket lining'] = pocket_lining
        
        article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
        if article_number_element:
            article_number_text = article_number_element.text.strip()
            article_number = re.search(r'\d+', article_number_text).group()
            product_info['Article number'] = article_number    
        
        # Adicionar as informações do produto à lista
        all_products_info.append(product_info)
        
    # Garante the same number o columns    
    df_composition = pd.concat([ df_patter, df_composition], axis=0)
    
    # criar data frame 
    df_composition = pd.DataFrame(all_products_info)

    df_composition['style_id'] = df_composition['Article number'].apply( lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Article number'].apply( lambda x: x[-3:])
    
    # guardo aqui minhas col de composition e concateno a lista somando
    aux = aux + df_composition.columns.tolist()

    # ========================== df_color + df_composition ================================== 
    if 'Size' in df_composition.columns:
        data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size']], how='left', on='style_id')
    else:
        data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')

    
    # data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size']], how='left', on='style_id')
    
    df_final = pd.concat([df_final, data_sku], axis=0)
    
# Join showroom data + df_final
    
data['style_id'] = data['product_id'].apply( lambda x: x[:-3])
data['color_id'] = data['product_id'].apply( lambda x: x[-3:])

data_raw = pd.merge(data, df_final[['style_id','color_name', 'Fit', 'Composition', 'Size']], how='left', on='style_id')    

https://www2.hm.com/en_us/productpage.1130309005.html


https://www2.hm.com/en_us/productpage.1130309004.html
https://www2.hm.com/en_us/productpage.0979945001.html
https://www2.hm.com/en_us/productpage.0985159001.html
https://www2.hm.com/en_us/productpage.1008549001.html
https://www2.hm.com/en_us/productpage.1024256001.html
https://www2.hm.com/en_us/productpage.1182102002.html
https://www2.hm.com/en_us/productpage.0979945003.html
https://www2.hm.com/en_us/productpage.1008549005.html
https://www2.hm.com/en_us/productpage.1159764002.html
https://www2.hm.com/en_us/productpage.1024256011.html
https://www2.hm.com/en_us/productpage.1024256006.html
https://www2.hm.com/en_us/productpage.1202825001.html
https://www2.hm.com/en_us/productpage.1096385001.html
https://www2.hm.com/en_us/productpage.1130309007.html
https://www2.hm.com/en_us/productpage.1008549006.html
https://www2.hm.com/en_us/productpage.1008549029.html
https://www2.hm.com/en_us/productpage.1182102001.html
https://www2.hm.com/en_us/productpage.1008110019.html
https://www2.hm.com/en_us/pr

In [165]:
df_final.sample(200)

,product_id,color_name,style_id,color_id,Fit,Composition,Size
0,1130309001,Denim black,1130309,001,Loose fit,Cotton 100%,"The model is 187cm/6'2"" and wears a size 31/32"
4,0875105011,Light denim blue,0875105,011,Relaxed fit,Cotton 100%,"The model is 187cm/6'2"" and wears a size 31/32"
2,0811993003,Dark denim blue,0811993,003,Regular fit,"Cotton 99%, Spandex 1%","The model is 186cm/6'1"" and wears a size 31/32"
19,0875105046,Dark denim blue,0875105,046,Relaxed fit,Cotton 100%,"The model is 185cm/6'1"" and wears a size 31/32"
1,0979945002,Denim blue,0979945,002,Loose fit,Cotton 100%,"The model is 187cm/6'2"" and wears a size 31/32"
...,...,...,...,...,...,...,...
0,0875105001,Denim blue,0875105,001,Relaxed fit,Cotton 100%,"The model is 187cm/6'2"" and wears a size 31/32"
5,0979945019,Black/patterned,0979945,019,Loose fit,Cotton 100%,"The model is 187cm/6'2"" and wears a size 31/32"
4,1024256006,Dark denim blue,1024256,006,Slim fit,"Cotton 99%, Spandex 1%",NaN
17,0811993048,Dark blue,0811993,048,Regular fit,"Cotton 99%, Spandex 1%","The model is 185cm/6'1"" and wears a size 31/32"
